# Open Weather Map
Exploration of openweathermap.org API.

## Prerequisits
- load libraries
- load configuration file to get API access data
- create URL for API access
- access API
- print general information

In [32]:
import toml
import requests
import pandas as pd

def del_lst(x):
    return x[0]

conf = toml.load('../../conf.toml')
owm = conf['owm']

url = 'https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lng}' \
      '&units=metric&APPID={key}'
url = url.format(lat=owm['lat'], lng=owm['lng'], key=owm['key'])

response = requests.get(url)
dic = response.json()

# keys = ['lat', 'lon', 'timezone', 'timezone_offset']
# gi = pd.DataFrame(list(map(dic.get, keys)), keys, ['General Information']).T
# gi['timezone_offset'] = pd.to_timedelta(gi.timezone_offset, 's')
# gi

## Current Weather
- Timezone is UTC
- temp, feels_like, dew_point: °C
- pressure: hPa
- humidity, clouds: %
- uvi: UV Index
- visibility: m
- wind_speed: m / s
- wind_deg: °

In [33]:
cur = pd.DataFrame(dic['current'])

weather = cur.weather[0]
cur[list(weather.keys())] = weather.values()
cur = cur.drop(columns='weather')

time = ['dt', 'sunrise', 'sunset']
cur[time] = cur[time].apply(pd.to_datetime, unit='s')
cur = cur.set_index('dt')

cur

,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,id,main,description,icon
dt,,,,,,,,,,,,,,,,
2021-05-30 22:58:17,2021-05-31 03:07:43,2021-05-31 19:46:33,9.12,9.12,1025,92,7.89,0,20,9000,1.03,360,801,Clouds,few clouds,02n


### Alerts

In [34]:
# ale = pd.DataFrame(dic['alerts'], ['german', 'english'])
#
# time = ['start', 'end']
# ale[time] = ale[time].apply(pd.to_datetime, unit='s')
#
# ale

## Minutely Data
- 1 hour
- 60 observations
- precipitation: mm

In [35]:
mnt = pd.DataFrame(dic['minutely'])
mnt = mnt.set_index('dt')
mnt.index = pd.to_datetime(mnt.index, unit='s')
mnt

,precipitation
dt,
2021-05-30 22:59:00,0
2021-05-30 23:00:00,0
2021-05-30 23:01:00,0
2021-05-30 23:02:00,0
2021-05-30 23:03:00,0
...,...
2021-05-30 23:55:00,0
2021-05-30 23:56:00,0
2021-05-30 23:57:00,0


## Hourly Data
- 2 days
- 48 observations
- wind_gust: metre/sec
- pop: %
- rain: mm

In [36]:
hr = pd.DataFrame(dic['hourly'])

hr['dt'] = pd.to_datetime(hr.dt, unit='s')
hr = hr.set_index('dt')

hr = hr.join(pd.DataFrame(hr.weather.apply(del_lst).tolist(), hr.index))
hr = hr.drop(columns='weather')
if 'rain' in hr.columns:
    hr.loc[hr.rain.isna(), 'rain'] = 0
    hr.loc[hr.rain.notna(), 'rain'] = hr.loc[hr.rain.notna(), 'rain'].apply(
        lambda x: x.get('1h')
    )
else:
    hr['rain'] = 0

hr

,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,pop,id,main,description,icon,rain
dt,,,,,,,,,,,,,,,,,
2021-05-30 22:00:00,8.99,7.73,1025,91,7.60,0.00,16,10000,2.36,37,2.41,0,801,Clouds,few clouds,02n,0
2021-05-30 23:00:00,9.12,7.91,1025,92,7.89,0.00,20,10000,2.32,37,2.38,0,801,Clouds,few clouds,02n,0
2021-05-31 00:00:00,9.03,7.86,1025,91,7.64,0.00,16,10000,2.25,32,2.32,0,801,Clouds,few clouds,02n,0
2021-05-31 01:00:00,8.97,7.77,1025,92,7.74,0.00,12,10000,2.28,39,2.38,0,801,Clouds,few clouds,02n,0
2021-05-31 02:00:00,8.97,7.70,1026,92,7.74,0.00,8,10000,2.37,48,2.42,0,800,Clear,clear sky,01n,0
2021-05-31 03:00:00,8.95,7.77,1026,94,8.04,0.00,4,10000,2.24,56,2.28,0,800,Clear,clear sky,01n,0
2021-05-31 04:00:00,9.43,8.24,1026,95,8.64,0.10,0,10000,2.37,68,2.46,0,800,Clear,clear sky,01d,0
2021-05-31 05:00:00,11.06,10.68,1026,94,10.11,0.38,0,10000,2.16,84,3.54,0,800,Clear,clear sky,01d,0
2021-05-31 06:00:00,12.61,12.25,1026,89,10.85,0.97,0,10000,2.61,98,3.75,0,800,Clear,clear sky,01d,0


## Daily Data
- 1 week
- 8 observations

In [37]:
time = ['dt', 'sunrise', 'sunset']
dly = pd.DataFrame(dic['daily'])
dly[time] = dly[time].apply(pd.to_datetime, unit='s')

dly = dly.join(pd.DataFrame(dly.temp.to_list()))
dly = dly.join(pd.DataFrame(dly.feels_like.to_list()).add_suffix('_feels_like'))
dly = dly.join(pd.DataFrame(dly.weather.apply(del_lst).to_list()))

dly = dly.drop(columns=['temp', 'feels_like', 'weather'])
dly = dly.set_index('dt')
dly

,sunrise,sunset,moonrise,moonset,moon_phase,pressure,humidity,dew_point,wind_speed,wind_deg,...,eve,morn,day_feels_like,night_feels_like,eve_feels_like,morn_feels_like,id,main,description,icon
dt,,,,,,,,,,,,,,,,,,,,,
2021-05-31 11:00:00,2021-05-31 03:07:43,2021-05-31 19:46:33,1622419680,1622448600,0.69,1025,61,10.33,4.09,61,...,16.85,11.06,17.49,10.54,16.57,10.68,800,Clear,clear sky,01d
2021-06-01 11:00:00,2021-06-01 03:06:48,2021-06-01 19:47:46,1622507520,1622539920,0.72,1021,69,12.65,4.00,43,...,15.32,12.09,18.06,9.21,14.87,11.63,803,Clouds,broken clouds,04d
2021-06-02 11:00:00,2021-06-02 03:05:56,2021-06-02 19:48:57,1622595000,1622631000,0.75,1018,70,13.63,6.29,123,...,18.94,10.36,19.19,16.36,18.93,9.80,801,Clouds,few clouds,02d
2021-06-03 11:00:00,2021-06-03 03:05:07,2021-06-03 19:50:05,1622682300,1622721900,0.79,1024,64,14.31,6.60,123,...,19.52,14.80,21.37,14.78,19.59,14.56,802,Clouds,scattered clouds,03d
2021-06-04 11:00:00,2021-06-04 03:04:21,2021-06-04 19:51:11,1622769420,1622812620,0.82,1027,67,15.05,5.33,137,...,21.02,14.89,21.35,15.50,21.21,14.60,804,Clouds,overcast clouds,04d
2021-06-05 11:00:00,2021-06-05 03:03:38,2021-06-05 19:52:14,1622856540,1622903220,0.85,1026,76,15.64,2.40,347,...,18.52,16.57,20.24,15.34,18.70,16.48,500,Rain,light rain,10d
2021-06-06 11:00:00,2021-06-06 03:02:58,2021-06-06 19:53:14,1622943600,1622993940,0.88,1022,58,12.80,5.79,59,...,18.31,15.69,21.07,14.95,18.18,15.53,800,Clear,clear sky,01d
2021-06-07 11:00:00,2021-06-07 03:02:22,2021-06-07 19:54:12,1623030780,1623084540,0.91,1021,69,15.19,5.89,47,...,18.00,15.72,21.22,14.86,18.02,15.59,500,Rain,light rain,10d
